In [ ]:
!wget https://github.com/karolpiczak/ESC-50/archive/refs/heads/master.zip
zip_ref = zipfile.ZipFile("/content/master.zip", 'r')
zip_ref.extractall("/tmp")
zip_ref.close()
dataSetLoc = '/tmp/ESC-50-master'

Mounted at /content/drive/


In [8]:
import zipfile
from google.colab import drive
drive.mount('/content/drive/')

zip_ref = zipfile.ZipFile("/content/drive/My Drive/ESC-50-master.zip", 'r')
zip_ref.extractall("/tmp")
zip_ref.close()
dataSetLoc = '/tmp/ESC-50-master'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
def features_extractor(file_path):
  audio, sample_rate = librosa.load(file_path, res_type='kaiser_fast')
  mfccs_features = librosa.feature.mfcc(y=audio, sr = sample_rate, n_mfcc=input_size)
  mfccs_scaled_features = np.mean(mfccs_features.T, axis = 0)

  return mfccs_scaled_features

import pandas as pd

input_size = 128

csvLoc = dataSetLoc + '/meta/esc50.csv'
metadata = pd.read_csv(csvLoc)

import numpy as np
import os
import librosa
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler

extracted_features = []
xx = []
yy = []
for index_num,row in tqdm(metadata.iterrows()):
  # if index_num > 1:
  #   break
  file_name = os.path.join(dataSetLoc + '/audio', str(row["filename"])) 
  final_class_labels = row["category"]
  data = features_extractor(file_name)
  xx.append(data)
  yy.append(final_class_labels)

904it [01:36,  9.62it/s]

In [4]:
X = np.array(xx)
y = np.array(yy)

from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
y= to_categorical(labelencoder.fit_transform(y))

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics

num_labels=y.shape[1]

model=Sequential()
###first layer
model.add(Dense(units=input_size, activation='relu', input_shape=(input_size,)))
model.add(Dropout(0.5))
###second layer
model.add(Dense(units=input_size*2, activation='relu'))
model.add(Dropout(0.5))
# ###third layer
model.add(Dense(units=input_size*4, activation='relu'))
model.add(Dropout(0.5))

###final layer
model.add(Dense(num_labels))
model.add(Activation('softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')
## Trianing my model
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime 

num_epochs = 100
num_batch_size = 32

checkpointer = ModelCheckpoint(filepath='saved_models/audio_classification.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               16512     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 512)               131584    
                                                                 
 dropout_2 (Dropout)         (None, 512)               0         
                                                                 
 dense_3 (Dense)             (None, 50)                2

In [6]:
test_accuracy=model.evaluate(X_test,y_test,verbose=0)
print(test_accuracy[1])

name = '1-20736-A-18.wav'	
filename= dataSetLoc + '/audio/' + name
prediction_feature=features_extractor(filename)

for index_num,row in metadata.iterrows():
  if index_num > 30:
    break
  file_name = os.path.join(dataSetLoc + '/audio', str(row["filename"])) 
  final_class_labels = row["category"]
  data = features_extractor(file_name)
  classes_x=np.argmax(model.predict(np.array([data])),axis=1)
  predicted_class_str = labelencoder.inverse_transform(classes_x)
  print('')
  print(final_class_labels)
  print(predicted_class_str)
  print('---')

0.4000000059604645

dog
['sneezing']
---

chirping_birds
['toilet_flush']
---

vacuum_cleaner
['vacuum_cleaner']
---

vacuum_cleaner
['vacuum_cleaner']
---

thunderstorm
['thunderstorm']
---

thunderstorm
['thunderstorm']
---

door_wood_knock
['fireworks']
---

can_opening
['can_opening']
---

crow
['crow']
---

door_wood_knock
['door_wood_knock']
---

door_wood_knock
['door_wood_knock']
---

clapping
['clapping']
---

clapping
['clapping']
---

clapping
['clapping']
---

dog
['sneezing']
---

clapping
['clapping']
---

thunderstorm
['thunderstorm']
---

fireworks
['fireworks']
---

fireworks
['fireworks']
---

fireworks
['fireworks']
---

fireworks
['fireworks']
---

clapping
['clapping']
---

clapping
['clapping']
---

clapping
['clapping']
---

chainsaw
['chainsaw']
---

airplane
['train']
---

mouse_click
['can_opening']
---

pouring_water
['keyboard_typing']
---

train
['train']
---

sheep
['cow']
---

water_drops
['car_horn']
---
